In [1]:
from __future__ import print_function
import numpy as np
import cv2
import os
from os import listdir
from os.path import isfile, join

In [2]:
# motion deblur
def blur_edge(img, d=31):
    h, w  = img.shape[:2]
    img_pad = cv2.copyMakeBorder(img, d, d, d, d, cv2.BORDER_WRAP)
    img_blur = cv2.GaussianBlur(img_pad, (2*d+1, 2*d+1), -1)[d:-d,d:-d]
    y, x = np.indices((h, w))
    dist = np.dstack([x, w-x-1, y, h-y-1]).min(-1)
    w = np.minimum(np.float32(dist)/d, 1.0)
    return img*w + img_blur*(1-w)

def motion_kernel(angle, d, sz=65):
    kern = np.ones((1, d), np.float32)
    c, s = np.cos(angle), np.sin(angle)
    A = np.float32([[c, -s, 0], [s, c, 0]])
    sz2 = sz // 2
    A[:,2] = (sz2, sz2) - np.dot(A[:,:2], ((d-1)*0.5, 0))
    kern = cv2.warpAffine(kern, A, (sz, sz), flags=cv2.INTER_CUBIC)
    
    return kern

# out of focus deblur
def defocus_kernel(d, sz=65):
    kern = np.zeros((sz, sz), np.uint8)
    cv2.circle(kern, (sz, sz), d, 255, -1, cv2.LINE_AA, shift=1)
    kern = np.float32(kern) / 255.0
    return kern


In [3]:
# load the video
video_path = 'input.mp4'
video_capture = cv2.VideoCapture(video_path)
# get the video information
fps = video_capture.get(cv2.CAP_PROP_FPS)
size = (int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), 
        int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fnums = video_capture.get(cv2.CAP_PROP_FRAME_COUNT)

In [4]:
# save images
file_path = 'images'
if not os.path.exists(file_path):
    os.mkdir(file_path)
i = 0
times = 100
nums = 10
isOpened = video_capture.isOpened()
while isOpened:
    if i == nums * times:
        break
    if i % times != 0:
        i = i + 1
        success, frame = video_capture.read()
        continue
    else:
        i = i + 1

    success, frame = video_capture.read()
    if i // times + 1 < 10:
        file_name = "image0" + str(i // times + 1) + ".jpg"
    else:
        file_name = "image" + str(i // times + 1) + ".jpg"
    print(file_name)
    if success == True:
        cv2.imwrite(file_path + '/' + file_name, frame)
video_capture.release()
print('>>finished<<')

image01.jpg
image02.jpg
image03.jpg
image04.jpg
image05.jpg
image06.jpg
image07.jpg
image08.jpg
image09.jpg
image10.jpg
>>finished<<


In [5]:
# load images
img_path = './images'
files = [join(img_path,f) for f in listdir(img_path) if isfile(join(img_path,f))]
img = []
for i in range(len(files)):
    img.append(cv2.imread(files[i]))
    
print(len(img))

10


In [12]:
# motion deblur or out of focus deblur
def deblur_func(img, angle=180, d=10, snr=30, motion_func=True):
    angle = np.deg2rad(180)
    noise = 10**(-0.1 * snr)
    
    img_rgb = img[:,:,:].copy()
    img_bw = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    #deblur_output = frame[:,:,:].copy()
    img_r = np.zeros_like(img_bw)
    img_g = np.zeros_like(img_bw)
    img_b = np.zeros_like(img_bw)

    img_r = img_rgb[..., 0]
    img_g = img_rgb[..., 1]

    img_b = img_rgb[..., 2]

    img_rgb = np.float32(img_rgb)/255.0
    img_bw = np.float32(img_bw)/255.0
    img_r = np.float32(img_r)/255.0
    img_g = np.float32(img_g)/255.0
    img_b = np.float32(img_b)/255.0

    #cv2.imshow('input', img_rgb)

    # img_bw = blur_edge(img_bw)
    img_r = blur_edge(img_r)
    img_g = blur_edge(img_g)
    img_b = blur_edge(img_b)

    # IMG_BW = cv2.dft(img_bw, flags=cv2.DFT_COMPLEX_OUTPUT)
    IMG_R = cv2.dft(img_r, flags=cv2.DFT_COMPLEX_OUTPUT)
    IMG_G = cv2.dft(img_g, flags=cv2.DFT_COMPLEX_OUTPUT)
    IMG_B = cv2.dft(img_b, flags=cv2.DFT_COMPLEX_OUTPUT)


    if motion_func:
        psf = motion_kernel(angle, d)
    else:
        psf = defocus_kernel(d)
    #cv2.imshow('psf', psf)
    #print(psf)
    psf /= psf.sum()
    psf_pad = np.zeros_like(img_bw)
    kh, kw = psf.shape
    psf_pad[:kh, :kw] = psf
    PSF = cv2.dft(psf_pad, flags=cv2.DFT_COMPLEX_OUTPUT, nonzeroRows = kh)
    PSF2 = (PSF**2).sum(-1)
    iPSF = PSF / (PSF2 + noise)[...,np.newaxis]

    # RES_BW = cv2.mulSpectrums(IMG_BW, iPSF, 0)
    RES_R = cv2.mulSpectrums(IMG_R, iPSF, 0)
    RES_G = cv2.mulSpectrums(IMG_G, iPSF, 0)
    RES_B = cv2.mulSpectrums(IMG_B, iPSF, 0)


    # res_bw = cv2.idft(RES_BW, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT )
    res_r = cv2.idft(RES_R, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT )
    res_g = cv2.idft(RES_G, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT )
    res_b = cv2.idft(RES_B, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT )

    res_rgb = np.zeros_like(img_rgb)
    res_rgb[..., 0] = res_r
    res_rgb[..., 1] = res_g
    res_rgb[..., 2] = res_b

    # res_bw = np.roll(res_bw, -kh//2, 0)
    # res_bw = np.roll(res_bw, -kw//2, 1)
    res_rgb = np.roll(res_rgb, -kh//2, 0)
    res_rgb = np.roll(res_rgb, -kw//2, 1)

    #cv2.imshow('motion deblur', res_rgb)
    #cv2.waitKey(0)

    #cv2.destroyAllWindows()
    return res_rgb

In [13]:
def save_output(img, path, func, i):
    file_path = path + "/" + func
    if not os.path.exists(file_path):
        os.makedirs(file_path)
    
    if i + 1 < 10:
        file_name = func + "0" + str(i + 1) + ".jpg"
    else:
        file_name = func + str(i + 1) + ".jpg"
    print(file_name)
    cv2.imwrite(file_path + '/' + file_name, img * 255)

In [18]:
img_index = 0
angle = 180
d = 10
snr = 29
# motion deblur
while img_index < len(img):
    cur = img[img_index]
    output = deblur_func(cur, angle, d, snr)
    save_output(output, "output", "motion_deblur", img_index)
    img_index = img_index + 1

print('motion deblur finished.')

motion_deblur01.jpg
motion_deblur02.jpg
motion_deblur03.jpg
motion_deblur04.jpg
motion_deblur05.jpg
motion_deblur06.jpg
motion_deblur07.jpg
motion_deblur08.jpg
motion_deblur09.jpg
motion_deblur10.jpg
motion deblur finished.


In [17]:
# out of focus deblur
img_index = 0
d = 7
snr = 11
while img_index < len(img):
    cur = img[img_index]
    output = deblur_func(cur, angle, d, snr, False)
    save_output(output, "output", "defocus_deblur", img_index)
    img_index = img_index + 1

print('defocus deblur finished.')

defocus_deblur01.jpg
defocus_deblur02.jpg
defocus_deblur03.jpg
defocus_deblur04.jpg
defocus_deblur05.jpg
defocus_deblur06.jpg
defocus_deblur07.jpg
defocus_deblur08.jpg
defocus_deblur09.jpg
defocus_deblur10.jpg
defocus deblur finished.
